In [ ]:
import pandas as pd
import numpy as np
import re
from glob import glob
import os
import shutil
from tqdm import tqdm
from collections import Counter
import gc

# Field Description

# C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS

# C/A      = Control Area (A002)
# UNIT     = Remote Unit for a station (R051)
# SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)
# STATION  = Represents the station name the device is located at
# LINENAME = Represents all train lines that can be boarded at this station
#            Normally lines are represented by one character.  LINENAME 456NQR repersents train server for 4, 5, 6, N, Q, and R trains.
# DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND   
# DATE     = Represents the date (MM-DD-YY)
# TIME     = Represents the time (hh:mm:ss) for a scheduled audit event
# DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
#            1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
#            2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. 
# ENTRIES  = The comulative entry register value for a device
# EXIST    = The cumulative exit register value for a device

# Example:
# The data below shows the entry/exit register values for one turnstile at control area (A002) from 09/27/14 at 00:00 hours to 09/29/14 at 00:00 hours

# C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,00:00:00,REGULAR,0004800073,0001629137,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,04:00:00,REGULAR,0004800125,0001629149,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,08:00:00,REGULAR,0004800146,0001629162,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,12:00:00,REGULAR,0004800264,0001629264,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,16:00:00,REGULAR,0004800523,0001629328,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-27-14,20:00:00,REGULAR,0004800924,0001629371,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,00:00:00,REGULAR,0004801104,0001629395,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,04:00:00,REGULAR,0004801149,0001629402,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,08:00:00,REGULAR,0004801168,0001629414,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,12:00:00,REGULAR,0004801304,0001629463,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,16:00:00,REGULAR,0004801463,0001629521,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-28-14,20:00:00,REGULAR,0004801737,0001629555,
# A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,09-29-14,00:00:00,REGULAR,0004801836,0001629574,

In [ ]:
data_links = ['!wget http://web.mta.info/developers/'+re.findall('href="(.*?)"',x)[0] for x in '''xxxxxx'''.split('<br>')]

print('\n\n'.join(data_links))

In [ ]:
data_filepath_list = sorted(glob('/content/turnstile_*'))

raw_turnstile_data_folder = '/content/drive/MyDrive/nyc_data/subway/raw/turnstile'

for p in tqdm(data_filepath_list):
  shutil.move(p, p.replace('/content', raw_turnstile_data_folder))
data_filepath_list = [p.replace('/content', raw_turnstile_data_folder) for p in data_filepath_list]

data_filepath_list = [p.replace('/content', raw_turnstile_data_folder) for p in data_filepath_list]

In [ ]:
# Check if all files share the same schema / header
# first_rows = []
# for p in data_filepath_list:
#   first_rows.append(tuple(pd.read_csv(p, nrows = 1, header = None).values.tolist()[0]))
# Counter(first_rows).most_common(1)

### Post 2015

In [ ]:
data_filepath_list = sorted(glob(raw_turnstile_data_folder+'/turnstile_*.txt'))
common_header_data_filepath_list = []
for p in data_filepath_list:
  df = pd.read_csv(p, nrows = 0)
  if tuple(df.columns.tolist()) == ('C/A','UNIT','SCP','STATION','LINENAME','DIVISION','DATE','TIME','DESC','ENTRIES','EXITS                                                               '):
    common_header_data_filepath_list.append(p)

for year in [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]:
  print(year)
  this_year_data_filepath_list = [p for p in common_header_data_filepath_list if 'turnstile_'+str(year%100) in p]
  this_year_csv_path = raw_turnstile_data_folder+'/turnstile_'+str(year)+'.csv'
  os.system('cat '+' '.join(this_year_data_filepath_list) + ' > '+this_year_csv_path)
  df = pd.read_csv(this_year_csv_path, error_bad_lines=False)
  df = df[df['C/A']!='C/A']
  df = df.rename(columns={'EXITS                                                               ':'EXITS'})
  df['ENTRIES'] = df['ENTRIES'].apply(int)
  df['EXITS'] = df['EXITS'].apply(int)
  df.to_parquet(this_year_csv_path.replace('.csv','.parquet'), compression='gzip')
  del df
  gc.collect()

### Pre 2015

In [ ]:
# Field Description

# C/A,UNIT,SCP,DATE1,TIME1,DESC1,ENTRIES1,EXITS1,DATE2,TIME2,DESC2,ENTRIES2,EXITS2,DATE3,TIME3,DESC3,ENTRIES3,EXITS3,DATE4,TIME4,DESC4,ENTRIES4,EXITS4,DATE5,TIME5,DESC5,ENTRIES5,EXITS5,DATE6,TIME6,DESC6,ENTRIES6,EXITS6,DATE7,TIME7,DESC7,ENTRIES7,EXITS7,DATE8,TIME8,DESC8,ENTRIES8,EXITS8


# C/A = Control Area (A002)
# UNIT = Remote Unit for a station (R051)
# SCP = Subunit Channel Position represents an specific address for a device (02-00-00)
# DATEn = Represents the date (MM-DD-YY)
# TIMEn = Represents the time (hh:mm:ss) for a scheduled audit event
# DEScn = Represent the "REGULAR" scheduled audit event (occurs every 4 hours)
# ENTRIESn = The comulative entry register value for a device
# EXISTn = The cumulative exit register value for a device



# Example:
# The data below shows the entry/exit register values for one turnstile at control area (A002) from 03/21/10 at 00:00 hours to 03/28/10 at 20:00 hours


# A002,R051,02-00-00,03-21-10,00:00:00,REGULAR,002670738,000917107,03-21-10,04:00:00,REGULAR,002670738,000917107,03-21-10,08:00:00,REGULAR,002670746,000917117,03-21-10,12:00:00,REGULAR,002670790,000917166,03-21-10,16:00:00,REGULAR,002670932,000917204,03-21-10,20:00:00,REGULAR,002671164,000917230,03-22-10,00:00:00,REGULAR,002671181,000917231,03-22-10,04:00:00,REGULAR,002671181,000917231
# A002,R051,02-00-00,03-22-10,08:00:00,REGULAR,002671220,000917324,03-22-10,12:00:00,REGULAR,002671364,000917640,03-22-10,16:00:00,REGULAR,002671651,000917719,03-22-10,20:00:00,REGULAR,002672430,000917789,03-23-10,00:00:00,REGULAR,002672473,000917795,03-23-10,04:00:00,REGULAR,002672474,000917795,03-23-10,08:00:00,REGULAR,002672516,000917876,03-23-10,12:00:00,REGULAR,002672652,000917934
# A002,R051,02-00-00,03-23-10,16:00:00,REGULAR,002672879,000917996,03-23-10,20:00:00,REGULAR,002673636,000918073,03-24-10,00:00:00,REGULAR,002673683,000918079,03-24-10,04:00:00,REGULAR,002673683,000918079,03-24-10,08:00:00,REGULAR,002673722,000918171,03-24-10,12:00:00,REGULAR,002673876,000918514,03-24-10,16:00:00,REGULAR,002674221,000918594,03-24-10,20:00:00,REGULAR,002675082,000918671
# A002,R051,02-00-00,03-25-10,00:00:00,REGULAR,002675153,000918675,03-25-10,04:00:00,REGULAR,002675153,000918675,03-25-10,08:00:00,REGULAR,002675190,000918752,03-25-10,12:00:00,REGULAR,002675345,000919053,03-25-10,16:00:00,REGULAR,002675676,000919118,03-25-10,20:00:00,REGULAR,002676557,000919179,03-26-10,00:00:00,REGULAR,002676688,000919207,03-26-10,04:00:00,REGULAR,002676694,000919208
# A002,R051,02-00-00,03-26-10,08:00:00,REGULAR,002676735,000919287,03-26-10,12:00:00,REGULAR,002676887,000919607,03-26-10,16:00:00,REGULAR,002677213,000919680,03-26-10,20:00:00,REGULAR,002678039,000919743,03-27-10,00:00:00,REGULAR,002678144,000919756,03-27-10,04:00:00,REGULAR,002678145,000919756,03-27-10,08:00:00,REGULAR,002678155,000919777,03-27-10,12:00:00,REGULAR,002678247,000919859
# A002,R051,02-00-00,03-27-10,16:00:00,REGULAR,002678531,000919908,03-27-10,20:00:00,REGULAR,002678892,000919964,03-28-10,00:00:00,REGULAR,002678929,000919966,03-28-10,04:00:00,REGULAR,002678929,000919966,03-28-10,08:00:00,REGULAR,002678935,000919982,03-28-10,12:00:00,REGULAR,002679003,000920006,03-28-10,16:00:00,REGULAR,002679231,000920059,03-28-10,20:00:00,REGULAR,002679475,000920098

In [ ]:
raw_turnstile_data_folder = '/content/drive/MyDrive/nyc_data/subway/raw/turnstile'

data_filepath_list = sorted(glob(raw_turnstile_data_folder+'/turnstile_*.txt'))

In [ ]:
for year in [2010, 2011, 2012, 2013, 2014]:
  print(year)
  this_year_data_filepath_list = [p for p in data_filepath_list if 'turnstile_'+str(year%100) in p]
  this_year_csv_path = raw_turnstile_data_folder+'/turnstile_'+str(year)+'.csv'
  os.system('cat '+' '.join(this_year_data_filepath_list) + ' > '+this_year_csv_path)

  df = pd.read_csv(this_year_csv_path, error_bad_lines=False)
  df.columns = 'C/A,UNIT,SCP,DATE1,TIME1,DESC1,ENTRIES1,EXITS1,DATE2,TIME2,DESC2,ENTRIES2,EXITS2,DATE3,TIME3,DESC3,ENTRIES3,EXITS3,DATE4,TIME4,DESC4,ENTRIES4,EXITS4,DATE5,TIME5,DESC5,ENTRIES5,EXITS5,DATE6,TIME6,DESC6,ENTRIES6,EXITS6,DATE7,TIME7,DESC7,ENTRIES7,EXITS7,DATE8,TIME8,DESC8,ENTRIES8,EXITS8'.split(',')
  long_df = pd.DataFrame()
  for i in range(1,8+1):
    long_df = long_df.append( df[['C/A','UNIT','SCP','DATE'+str(i),'TIME'+str(i),'DESC'+str(i),'ENTRIES'+str(i),'EXITS'+str(i)]].rename(columns = {'DATE'+str(i):'DATE','TIME'+str(i):'TIME','DESC'+str(i):'DESC','ENTRIES'+str(i):'ENTRIES','EXITS'+str(i):'EXITS'}), ignore_index = True )
  long_df['ENTRIES'] = long_df['ENTRIES'].apply(str)
  long_df['EXITS'] = long_df['EXITS'].apply(str)
  long_df.to_parquet(this_year_csv_path.replace('.csv','.parquet'), compression='gzip')
  del long_df, df
  gc.collect()